In [1]:
import random
from collections import Counter

import numpy as np
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
with open('AnnaKarenina.txt', encoding='utf-8') as file:
    corpora = file.read()
    
with open('WarAndPeace.txt', encoding='utf-8') as file:
    test_corpora = file.readlines()
test_text = '.'.join(test_corpora[1991].split('.')[:10])
test_corpora = '\n'.join(test_corpora)

In [3]:
RUS_ALPHABET = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '

# 1. Unigrams Baseline

In [4]:
def process_text(text):
    """
    Обрабатываем текст, убирая всю пунктуацию и приводя символы к нижнему регистру.
    """
    return ''.join([char for char in text.lower() if char in RUS_ALPHABET])


def encode(text):
    """
    Шифруем текст случайной перестановкой символов.
    """
    mapping = dict(zip(RUS_ALPHABET, random.sample(RUS_ALPHABET, len(RUS_ALPHABET))))
    return ''.join([mapping[char] for char in text])


def decoding_accuracy(true, pred):
    """
    Точность (доля угаданных символов)
    """
    total = len(pred)
    correct = 0
    for i in range(total):
        if true[i] == pred[i]:
            correct += 1
    return correct / total * 100


class Decoder():
    def __init__(self, n_gram=1):
        self.n_gram = n_gram
    
    def fit(self, text):
        """
        Сортируем символы по частоте встречаемости.
        """
        processed_text = text
        if self.n_gram != 1:
            processed_text = [text[i: i + self.n_gram] for i in range(len(text) - self.n_gram)]
        self.mapping = [char[0] for char in Counter(processed_text).most_common()]
    
    def decode(self, text):
        """
        Декодируем текст.
        """
        processed_text = text
        if self.n_gram != 1:
            processed_text = [text[i: i + self.n_gram] for i in range(len(text) - self.n_gram)]
        most_freq = [char[0] for char in Counter(processed_text).most_common()]
        mapping = dict(zip(most_freq, self.mapping))
        decoded_text = []
        for i in range(0, len(text) - self.n_gram + 1, self.n_gram):
            decoded_text.append(mapping[text[i: i + self.n_gram]])
        return ''.join(decoded_text)

In [5]:
# Обучаем декодер на тексте романа "Анна Каренина"
train_corpora = process_text(corpora)
decoder = Decoder()
decoder.fit(train_corpora)

# Кодируем тестовый корпус текста
test_text = process_text(test_text)
encoded_test_text = encode(test_text)

print(f'Оригинальный текст\n\n{test_text}\n\n')
print(f'Закодированный текст\n\n{encoded_test_text}')

Оригинальный текст

прошло несколько минут суетни около высокой кровати люди несшие больного разошлись анна михайловна дотронулась до руки пьера и сказала ему  идите пьер вместе с нею подошел к кровати на которой в праздничной позе видимо имевшей отношение к толькочто совершенному таинству был положен больной он лежал высоко опираясь головой на подушки руки его были симметрично выложены на зеленом шелковом одеяле ладонями вниз когда пьер подошел граф глядел прямо на него но глядел тем взглядом которого смысл и значение нельзя понять человеку или этот взгляд ровно ничего не говорил как только то что покуда есть глаза надо же глядеть куданибудь или он говорил слишком многое пьер остановился не зная что ему делать и вопросительно оглянулся на свою руководительницу анну михайловну анна михайловна сделала ему торопливый жест глазами указывая на руку больного и губами посылая ей воздушный поцелуй


Закодированный текст

эчтхкт ёцпьткоьт нщёгю пгцюёщ тьткт миптьту ьчтмдющ кфжщ ёцпхщц вткоётрт

Теперь раскодируем тестовый текст, используя наш обученный декодер, и оценим качество расшифровки (доля угаданных символов).

In [6]:
decoded_test_text = decoder.decode(encoded_test_text)
accuracy = decoding_accuracy(test_text, decoded_test_text)

print(f'Расшифрованный текст\n\n{decoded_test_text}\n\nAccuracy: {accuracy:.2f} %')

Расшифрованный текст

ьрочно еаквонпво утедс кдасет овоно лзковоб вролист нхмт еакчта йонпеояо ригочнткп иееи утюибнолеи мосроедникп мо рдвт ьпари т квигини ауд  тмтса ьпар луакса к еах ьомочан в вролист еи восороб л ьригметжеоб ьога лтмтуо туалчаб осеочаета в сонпвожсо коларчаееоуд ситекслд йзн ьоношае йонпеоб ое нашин лзково оьтриыкп яонолоб еи ьомдчвт рдвт аяо йзнт ктууасртжео лзношаез еи ганаеоу чанволоу омаына нимоеыут летг воями ьпар ьомочан ярищ яныман ьрыуо еи еаяо ео яныман сау лгянымоу восорояо кузкн т геижаета еанпгы ьоеысп жанолавд тнт цсос лгяным ролео етжаяо еа яолортн вив сонпво со жсо ьовдми аксп яниги еимо ша янымасп вдмиетйдмп тнт ое яолортн кнтчвоу уеояоа ьпар оксиеолтнкы еа геиы жсо ауд манисп т лоьроктсанпео ояныеднкы еи клох рдволомтсанпетэд иеед утюибнолед иееи утюибнолеи кманини ауд сороьнтлзб шакс янигиут двигзлиы еи рдвд йонпеояо т ядйиут ьокзниы аб логмдчезб ьоэандб

Accuracy: 30.12 %


Качество декодирования оставляет желать лучшего, текст абсолютно нечитабелен, точность чуть выше 30 %. Это можно объяснить тем, что тестовый корпус имеет небольшой размер и на нем нет возможности получить правильное распредление частот символов.

Например, если мы возьмем в качестве тестового текста роман "Война и мир" целиком, а не только его абзац, то увидим, что качество значительно улучшилось, можно даже угадать отдельные слова.

In [7]:
test_corpora = process_text(test_corpora)

decoded_test_text = decoder.decode(encode(test_corpora))
accuracy = decoding_accuracy(test_corpora, decoded_test_text)

print(f'Оригинальный текст\n\n{test_corpora[:1000]}\n\n')
print(f'Расшифрованный текст\n\n{decoded_test_text[:1000]}\n\nAccuracy: {accuracy:.2f} %')

Оригинальный текст

война и мир  самый известный роман льва николаевича толстого как никакое другое произведение писателя отражает глубину его мироощущения и философииэта книга из разряда вечных потому что она обо всем  о жизни и смерти о любви и чести о мужестве и героизме о славе и подвиге о войне и мирепервый том знакомит с высшим обществом россии  века показаны взаимоотношения между родителями и детьми в семье ростовых сватовство у болконских интриги у безуховых вечера в салоне фрейлины апшерер балы в москве и петербурге  лев николаевич толстойчасть перваячасть ораячасть третья  лев николаевич толстойвойна и миртом часть первая е              поместья                                                       мой верный раб    ну что князь генуа и лукка стали не больше как поместьями фамилии бонапарте нет я вас предупреждаю если вы мне не скажете что у нас война если вы еще позволите себе защищать все гадости все ужасы этого антихриста право я верю что он антихрист  я вас больше не знаю

# 2. Bigrams Baseline

Порядок действий не меняется, но в этом случае в качестве отдельных токенов берем пару идущих друг за другом символов.

In [8]:
# Обучаем декодер на тексте романа "Анна Каренина"
decoder = Decoder(n_gram=2)
decoder.fit(train_corpora)

In [9]:
decoded_test_text = decoder.decode(encoded_test_text)
accuracy = decoding_accuracy(test_text, decoded_test_text)

print(f'Расшифрованный текст\n\n{decoded_test_text}\n\nAccuracy: {accuracy:.2f} %')

Расшифрованный текст

илолала ва вго вороеочонокмае ри ко се р впоед сите идчта вауд иасготона ботолу ль лт и рообче с пю ернов колью о вемо нодре донав ако чаттыамся идатая нном икувосоним тьканыед сите  п терк н я  нреаялавын  нар ишечтча дннийхоовмаолпрег тос к зсиь он статьт огстадоеоммнзаы  н киспрза клин овэтсьошкасе р вовньрелсеннаалл по п нандимо бтее муо ееу онс ннтр этоя ы испрьн пив опрогый о вл етанумсьдоожсксве  ету тратв нодменим тькаопгднееллокаилсво  па муо тонееллока  етизй виог терк рао ажескае ектнпрега  овр мниичиныл о сутстдее озеря дуелусно ео ламинаа  инал х каавныь го восо кио ни утв чоменй оти  пм ия ий виемен утвлажныв ду овэтнал х касабу ветуграую нодмесладтошеля мвоив п мкио акстлокоин дя имнобо  готоовй д иеужа и ыесн бте санся олилыстлев орвс гал естле порвс гал еи еш оойи аксть номычуняияважей отспе теотрябыа и ве уза клирао е учрьро нслы бы чпол аядизнпонигаба

Accuracy: 11.00 %


Качество декодирования стало значительно хуже. Скорее всего это связано с тем, что число различных биграмм выше, поэтому размер тестового корпуса влияет еще сильнее. Многие биграммы встречаются одинаково малое число раз и поэтому не отличаются друг от друга с точки зрения алгоритма. Увеличение тестового корпуса увеличивает точность, но текст все так же нечитабелен.

In [10]:
decoded_test_text = decoder.decode(encode(test_corpora))
accuracy = decoding_accuracy(test_corpora, decoded_test_text)

print(f'Оригинальный текст\n\n{test_corpora[:1000]}\n\n')
print(f'Расшифрованный текст\n\n{decoded_test_text[:1000]}\n\nAccuracy: {accuracy:.2f} %')

Оригинальный текст

война и мир  самый известный роман льва николаевича толстого как никакое другое произведение писателя отражает глубину его мироощущения и философииэта книга из разряда вечных потому что она обо всем  о жизни и смерти о любви и чести о мужестве и героизме о славе и подвиге о войне и мирепервый том знакомит с высшим обществом россии  века показаны взаимоотношения между родителями и детьми в семье ростовых сватовство у болконских интриги у безуховых вечера в салоне фрейлины апшерер балы в москве и петербурге  лев николаевич толстойчасть перваячасть ораячасть третья  лев николаевич толстойвойна и миртом часть первая е              поместья                                                       мой верный раб    ну что князь генуа и лукка стали не больше как поместьями фамилии бонапарте нет я вас предупреждаю если вы мне не скажете что у нас война если вы еще позволите себе защищать все гадости все ужасы этого антихриста право я верю что он антихрист  я вас больше не знаю

# 3. MCMC sampling

Давайте будем считать, что мы имеем распредление текстов опредленной длины, то есть в зависимости от того, из каких символов он состоит, он будет иметь разное правдоподобие. Используя MCMC метод, мы можем сэмплировать тексты из этого распредления. По ходу сэмплирования можно отобрать наиболее правдоподобный текст. Таким образом алгоритм действий следующий:
1. Необходимо сэплировать $x'$ из $q(x'|x_t)$, где $x_t$ - текст на предыдущем шаге. Это можно сделать следуюшим образом: выбрать случайным образом 2 буквы и поменять их местами в тексте $x_t$.


2. Теперь нам необходимо оценить параметр $a$ МСМС алгоритма: $$a = \frac{p(x')}{p(x_t)}\cdot\frac{q(x_t|x')}{q(x'|x_t)} \\ \\ $$ Так как буквы для замены выбираются равновероятно, $q(x_t|x')$ и $q(x'|x_t)$ будут равны и их можно сократить. $p(x')$ и $p(x_t)$ это оценка правдоподобия нового и предыдущего текстов соответственно. Их можно оценить, перемножив веротяности n-грамм, из которых составлен текст, а их в свою очередь мы можем определить по корпусу, на котором обучаемся.


3. Далее в соответсвии с МСМС алгоритмом мы принимаем $x'$ в качестве нового $x_{t+1}$, если $a>1$, в противном случае принимаем $x'$ с вероятностью $a$, а с вероятностью $1-a$ оставляем $x_t$.

Итеративно повторяем алгоритм несколко раз и выбираем наиболее правдоподобный текст.

In [11]:
class MCMCDecoder():
    def __init__(self, n_gram=1):
        self.n_gram = n_gram
        self.nan = 1 / len(RUS_ALPHABET) ** n_gram
        
    def fit(self, text):
        processed_text = text
        if self.n_gram != 1:
            processed_text = [text[i: i + self.n_gram] for i in range(len(text) - self.n_gram)]
        counted = Counter(processed_text)
        total = sum(counted.values())
        self.mapping = {char: freq / total for char, freq in counted.items()}
    
    def _log_like(self, text):
        processed_text = text
        if self.n_gram != 1:
            processed_text = [text[i: i + self.n_gram] for i in range(len(text) - self.n_gram)]
        result = 0
        for n_gram, count in Counter(processed_text).items():
            result += count * np.log(self.mapping.get(n_gram, self.nan))
        return result
    
    def _sample_text(self, text):
        letter_1, letter_2 = np.random.choice(list(RUS_ALPHABET), 2, replace=False)
        sampled_text = []
        for letter in text:
            if letter == letter_1:
                sampled_text.append(letter_2)
            elif letter == letter_2:
                sampled_text.append(letter_1)
            else:
                sampled_text.append(letter)
        return ''.join(sampled_text)
    
    def _switch(self, a):
        if a > 1:
            return True
        return np.random.uniform() < a

    def decode(self, text, n_iter=100, orig_text=None, verbose_per=-1, print_text=False):
        curr_text = text
        best_text = curr_text
        curr_log_like = self._log_like(best_text)
        best_log_like = curr_log_like
        for i in tqdm(range(n_iter)):
            new_text = self._sample_text(curr_text)
            new_log_like = self._log_like(new_text)
            a = np.exp(new_log_like - curr_log_like)
            if self._switch(a):
                curr_text = new_text
                curr_log_like = new_log_like
                if curr_log_like > best_log_like:
                    best_text = curr_text
                    best_log_like = curr_log_like
                    
            if verbose_per > 0:
                if (i + 1) % verbose_per == 0:
                    if orig_text is not None:
                        accuracy = decoding_accuracy(orig_text, best_text)
                        print(f'Accuracy: {accuracy:.2f} %')
                        
                    print(f'Правдоподобие: {best_log_like:.2f}\n\n')
                    if print_text:
                        print(f'{best_text}\n\n')
        return best_text

In [12]:
decoder = MCMCDecoder(n_gram=2)
decoder.fit(train_corpora)

In [13]:
decoded_text = decoder.decode(
    encoded_test_text,
    n_iter=10000,
    orig_text=test_text,
    verbose_per=2000,
    print_text=True
)

 21%|████████████████▎                                                           | 2147/10000 [00:02<00:07, 987.05it/s]

Accuracy: 37.03 %
Правдоподобие: -5416.11


гкойтоен ъдотвдоеминусеъу сниеодотоерёъодожедкорасиетблиен ъйи ехотвнопоеказойтиъвеаннаемичажторнаелосконутаъвелоекудиегв каеиеъдазатае муееилис егв керм ъс еъен беголой тедедкорасиенаедосокожерегказлниьножегоз ерилимоеим рй жеосной ни едесотвдоьсоеъор кй нномуесаинъсруехётеготоя нехотвножеонет яатерёъодоеогикаыъвепотороженаеголуйдиекудие поехётиеъимм скиьноерётоя нёенаез т номей тдоромеол ыт еталонымиернизедоплаегв кеголой тепкашептыл тегкымоенаен поеноептыл тес мерзптыломедосокопоеъмёътеиезнаь ни ен твзыегонысвеь тор дуеитиещсосерзптылекорноениь поен епорокитедадесотвдоесоеьсоегодулае ъсвептазаеналоея ептыл сведуланихулвеитиеонепорокитеътийдомемнопо егв кеоъсаноритъыен езнаыеьсое муел тасвеиерогкоъис твноеоптынутъыенаеъробекудоролис твницуеаннуемичажторнуеаннаемичажторнаеъл татае муесокогтирёжея ъсептазамиеудазёраыенаекудуехотвнопоеиепухамиегоъётаые жерозлуйнёжегоц туж




 42%|████████████████████████████████▏                                           | 4237/10000 [00:04<00:05, 976.65it/s]

Accuracy: 84.94 %
Правдоподобие: -4951.87


прочло нестолыто минук суекни отоло вёсотой троваки льди несчие холыного разочлисы анна миъайловна докронуласы до рути пыера и стазала ему  идике пыер вмеске с неь подочел т троваки на токорой в празднижной позе видимо имевчей окночение т колытожко соверченному каинскву хёл полобен холыной он лебал вёсото опираясы головой на подучти рути его хёли симмекрижно вёлобенё на зеленом челтовом одеяле ладонями вниз тогда пыер подочел граю глядел прямо на него но глядел кем взглядом токорого смёсл и знажение нелызя понякы желовету или фкок взгляд ровно нижего не говорил тат колыто ко жко потуда ескы глаза надо бе глядекы туданихуды или он говорил сличтом многое пыер оскановился не зная жко ему делакы и вопросикелыно оглянулся на своь рутоводикелынишу анну миъайловну анна миъайловна сделала ему коропливёй беск глазами утазёвая на руту холыного и гухами посёлая ей воздучнёй пошелуй




 62%|██████████████████████████████████████████████▌                            | 6204/10000 [00:06<00:03, 1025.99it/s]

Accuracy: 99.32 %
Правдоподобие: -4844.99


прошло несколько минут суетни около высокой кровати люди несшие больного разошлись анна миъайловна дотронулась до руки пьера и сказала ему  идите пьер вместе с нею подошел к кровати на которой в праздничной позе видимо имевшей отношение к толькочто совершенному таинству был положен больной он лежал высоко опираясь головой на подушки руки его были симметрично выложены на зеленом шелковом одеяле ладонями вниз когда пьер подошел грах глядел прямо на него но глядел тем взглядом которого смысл и значение нельзя понять человеку или этот взгляд ровно ничего не говорил как только то что покуда есть глаза надо же глядеть куданибудь или он говорил слишком многое пьер остановился не зная что ему делать и вопросительно оглянулся на свою руководительниёу анну миъайловну анна миъайловна сделала ему торопливый жест глазами указывая на руку больного и губами посылая ей воздушный поёелуй




 82%|█████████████████████████████████████████████████████████████▌             | 8207/10000 [00:08<00:01, 1052.49it/s]

Accuracy: 99.32 %
Правдоподобие: -4844.99


прошло несколько минут суетни около высокой кровати люди несшие больного разошлись анна миъайловна дотронулась до руки пьера и сказала ему  идите пьер вместе с нею подошел к кровати на которой в праздничной позе видимо имевшей отношение к толькочто совершенному таинству был положен больной он лежал высоко опираясь головой на подушки руки его были симметрично выложены на зеленом шелковом одеяле ладонями вниз когда пьер подошел грах глядел прямо на него но глядел тем взглядом которого смысл и значение нельзя понять человеку или этот взгляд ровно ничего не говорил как только то что покуда есть глаза надо же глядеть куданибудь или он говорил слишком многое пьер остановился не зная что ему делать и вопросительно оглянулся на свою руководительниёу анну миъайловну анна миъайловна сделала ему торопливый жест глазами указывая на руку больного и губами посылая ей воздушный поёелуй




100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:09<00:00, 1014.95it/s]

Accuracy: 99.32 %
Правдоподобие: -4844.99


прошло несколько минут суетни около высокой кровати люди несшие больного разошлись анна миъайловна дотронулась до руки пьера и сказала ему  идите пьер вместе с нею подошел к кровати на которой в праздничной позе видимо имевшей отношение к толькочто совершенному таинству был положен больной он лежал высоко опираясь головой на подушки руки его были симметрично выложены на зеленом шелковом одеяле ладонями вниз когда пьер подошел грах глядел прямо на него но глядел тем взглядом которого смысл и значение нельзя понять человеку или этот взгляд ровно ничего не говорил как только то что покуда есть глаза надо же глядеть куданибудь или он говорил слишком многое пьер остановился не зная что ему делать и вопросительно оглянулся на свою руководительниёу анну миъайловну анна миъайловна сделала ему торопливый жест глазами указывая на руку больного и губами посылая ей воздушный поёелуй




Видно, что текст декодируется практически идеально.  Однако, стоит отметить, что в некоторых случаях алгоритм может сходиться очень долго, все зависит от того, насколько удачно происходит замена букв.

# 4. Расшифровка сообщения

In [12]:
message = 'დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ'

Для начала 'переведем' сообщение на русский, просто заменив символы на русские буквы случайным образом.

In [13]:
message_alphabet = list(set(message))
mapping = dict(zip(message_alphabet, RUS_ALPHABET))
rus_message = ''.join([mapping[char] for char in message])
print(f'Сообщение\n\n{rus_message}')

Сообщение

ъпйжхшрхшжцжкъхмощбёйамрчхжйжхсофкжхмощбёйамрчхкъдпкхлхвкотохпоониъмжухдокощрчхйътдохсщофжкёкахпдощъъхшпътохшрхшпъхпцъйёйжхсщёшжйамохжхсойлфжкъхбёдпжбёйамрчхнёййхеёхсопйъцмъъхфъкшъщкоъхеёцёмжъхдлщпёхгокухдомъфмохухмжфътохмъхонъиёз


In [ ]:
decoder = MCMCDecoder(n_gram=4)
decoder.fit(train_corpora)

decoded_text = decoder.decode(
    rus_message,
    n_iter=10000,
    verbose_per=1000,
    print_text=True
)

 12%|█████████▍                                                                  | 1235/10000 [00:01<00:11, 754.08it/s]

Правдоподобие: -2636.76


есив да двйвте норуюизная вив почтв норуюизная текст ц ётого сообъенвь которая иегко прочвтютз скорее дсего да дсе сйеиюив прюдвизно в поицчвте уюксвуюизная бюии лю посиейнее четдертое люйюнве кцрсю моть конечно ь нвчего не обеъюы




 22%|████████████████▍                                                           | 2158/10000 [00:02<00:09, 830.54it/s]

Правдоподобие: -2197.74


если вы визите нордальный или почти нордальный текст ю штого сообщения который легко прочитать скорее всего вы все сзелали правильно и полючите даксидальный балл ма послезнее четвертое мазание кюрса хотя конечно я ничего не обещаё




 31%|███████████████████████▋                                                    | 3122/10000 [00:03<00:08, 801.26it/s]

...

Правдоподобие: -2097.39


если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаш




 91%|████████████████████████████████████████████████████████████████████▉       | 9065/10000 [00:11<00:01, 733.77it/s]

Правдоподобие: -2097.39


если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаш




100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:12<00:00, 775.42it/s]

Правдоподобие: -2097.39


если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаш




Сообщение расшифровано идеально (одна ошибка в слове **обещаш**). Такого результата удалось добиться только при n_gram=4. При использовании биграмм тоже получалось что-то читабельное, но гораздо дальше от идеала. Алгоритм нестабилен, сойтись к чему-то нормальному удается не с первой попытки, как правило происходит застревание на какой-либо комбинации букв. Также стоит отметить, что чем больше закодированный текст, тем легче алгоритму сойтись к оптимальному решению.

# 5. Higher n_grams

In [12]:
decoder = MCMCDecoder(n_gram=2)
decoder.fit(train_corpora)
decoded_text_2_gram = decoder.decode(
    encoded_test_text,
    n_iter=10000,
    verbose_per=-1,
    print_text=False
)

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:24<00:00, 415.64it/s]


In [16]:
decoder = MCMCDecoder(n_gram=3)
decoder.fit(train_corpora)
decoded_text_3_gram = decoder.decode(
    encoded_test_text,
    n_iter=10000,
    verbose_per=-1,
    print_text=False
)

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:38<00:00, 257.31it/s]


In [18]:
decoder = MCMCDecoder(n_gram=4)
decoder.fit(train_corpora)
decoded_text_4_gram = decoder.decode(
    encoded_test_text,
    n_iter=10000,
    verbose_per=-1,
    print_text=False
)

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:47<00:00, 209.83it/s]


In [21]:
decoder = MCMCDecoder(n_gram=5)
decoder.fit(train_corpora)
decoded_text_5_gram = decoder.decode(
    encoded_test_text,
    n_iter=10000,
    verbose_per=-1,
    print_text=False
)

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:51<00:00, 195.69it/s]


In [23]:
texts = [decoded_text_2_gram, decoded_text_3_gram, decoded_text_4_gram, decoded_text_5_gram]
n_grams = list(range(2, 6))
for n_gram, text in zip(n_grams, texts):
    print(f'Точность при n_gram={n_gram}: {decoding_accuracy(test_text, text):.2f} %')

Точность при n_gram=2: 99.32 %
Точность при n_gram=3: 99.89 %
Точность при n_gram=4: 100.00 %
Точность при n_gram=5: 100.00 %


Основные выводы:
- При большем числе n_gram можно получить большую максимальную точность. Для маленького числа n_gram не удалось достичь идеального декодирования.
- При одном и том же числе итераций алгоритм приходит к оптимуму примерно с одной и той же частотой для разного числа n_gram.
- Для больших значений n_gram как правило не бывает промежуточных состояний, т.е. точность либо около 0, либо около 100 %. Для маленького числа n_gram могут быть любые значения, т.е. алгоритм сходиться более плавно.
- Самым стабильным оказался вариант с n_gram=4. Он выдает максималную точность, но в сравнении с n_gram=5 чаще сходится к оптимуму.